## Цель работы: ознакомление с методами поиска минимума функции двух переменных в оптимизационных задачах без ограничений (метод ГауссаЗейделя, метод наискорейшего спуска, методы сопряженных направлений).

## Вариант 10
$$
f(x) =20-(x-1)*e^{-x+1} - (y-2)*e^{-y+2}, a = -10, b = 10
$$

In [32]:
import math
import numpy as np

def dihotomy(a, b, func, eps = 0.001):
    while (abs(b-a) > 2*eps):
        L = (a+b)/2
        x1 = L - (eps/2)
        x2 = L + (eps/2)
        
        if func(x1)>func(x2):
            a = x1
        else:
            b = x2  
    result = (a+b)/2

    return result

def golden_ratio(a, b, func, tau = 0.618, eps = 0.001):
        L = b - a
        f1, f2 = 0, 0

        while(L > eps):
            delta =  L*tau
            x1, x2 = a + delta, b - delta
            f1, f2 = func(x1), func(x2)
    
            if f1 > f2:
                b = x1
                f1, x1 = f2, x2
                L = b - a
                x2, f2 = a + b - x1, func(x2)
    
            else:
                a = x2
                f2, x2 = f1, x1
                L = b - a
                x1, f1 = b - (x2 - a), func(x1)
                
        return (a+b)/2

In [33]:
def Gauss(x, a, b, func, n=2, eps = 0.001):
    y = x
    j = 1
    e = np.eye(n)
    while(True):
        f = lambda x: func(y+x*e[j-1])
        lmbd = dihotomy(a, b, f)

        y = y + lmbd*e[j-1]
        if j == n:
            x_pred = x
            x = y
            if np.linalg.norm(x - x_pred)<eps:
                break
            else:
                j=1
        else:
            j+=1

    return x

In [34]:
def Cauchy(x, a, b, func, grad, eps = 0.001):
    while(True):
        if np.linalg.norm(grad(x)) < eps:
            break
        else:
            S = -(grad(x) / np.linalg.norm(grad(x)))
            f = lambda k: func(x+k*S)
            lmbd = golden_ratio(a, b, f)
            x = x + lmbd*S

    return x

In [35]:
def HD(x, a, b, func, n=2, eps = 0.001):
    y = x
    j = 1
    e = np.eye(n)
    while(True):
        f = lambda x: func(y+x*e[j-1])
        lmbd = dihotomy(a, b, f)
        y = y + lmbd*e[j-1]
        if j == n:
            x_pred = x
            x = y
            if np.linalg.norm(x - x_pred)<eps:
                break
            else:
                d = x - x_pred
                j=1
                f = lambda x: func(y+x*d)
                lmbd = dihotomy(a, b, f)
                y = y + lmbd*d

        else:
            j+=1

    return x

In [36]:
def Rozenbroke(x, a, b, func, n=2, eps = 0.001):
    y = x
    j = 1
    e = np.eye(n)
    while(True):
        f = lambda x: func(y+x*e[j-1])
        lmbd = dihotomy(a, b, f)
        
        if lmbd>=0:
            y = y + lmbd*e[j-1]
            if j==n:
                x_pred = x 
                x = y
                if np.linalg.norm(x - x_pred)<eps:
                    return x

                else:
                    g1 = e[0]
                    g2 = e[1] - ((np.dot(e[1], g1)) / (np.linalg.norm(g1))**2) * g1
                    e = np.array([g1, g2])
        else:
            j+=1

In [37]:
x = [1, 1]
a = -10
b = 10
func = lambda x: 20-(x[0]-1)*math.exp(-(x[0]-1))-(x[1]-2)*math.exp(-(x[1]-2))
dfdx = lambda x: (x[0]-1)*np.exp(-(x[0]-1)) - np.exp(-(x[0]-1))
dfdy = lambda x: (x[1]-2)*np.exp(-(x[1]-2)) - np.exp(-(x[1]-2))
grad = lambda x: np.array([dfdx(x), dfdy(x)])

In [38]:
import pandas as pd
df = pd.DataFrame(
    {'Gauss':Gauss(x, a, b, func), 
     'Cauchy':Cauchy(x, a, b, func, grad), 
     'Hooke-Jeeves':HD(x, a, b, func), 
     'Rozenbroke':Rozenbroke(x, a, b, func),
    }, index = ['x', 'y'])

In [39]:
df

,Gauss,Cauchy,Hooke-Jeeves,Rozenbroke
x,1.999706,1.997329,2.000011,2.000011
y,3.000022,3.000334,3.000022,3.000022
